<a href="https://colab.research.google.com/github/skj092/Image-Classification-with-CIFAR-100/blob/main/ResnetPretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [2]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar100.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

169017344/169001437 [==============================] - 2s 0us/step


In [3]:
from tensorflow import keras

base_model = keras.applications.DenseNet169(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(32, 32, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

51888128/51877672 [==============================] - 1s 0us/step


In [4]:
base_model.trainable = False

In [5]:
inputs = keras.Input(shape=(32, 32, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(100)(x)
model = keras.Model(inputs, outputs)

In [6]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

%time model.fit(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels))

model.evaluate(test_images,  test_labels, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 136s 64ms/step - loss: 2.9148 - accuracy: 0.3009 - val_loss: 2.5980 - val_accuracy: 0.3543
Epoch 2/5
1563/1563 [==============================] - 94s 60ms/step - loss: 2.3044 - accuracy: 0.4135 - val_loss: 2.5174 - val_accuracy: 0.3745
Epoch 3/5
1563/1563 [==============================] - 94s 60ms/step - loss: 2.1131 - accuracy: 0.4555 - val_loss: 2.4835 - val_accuracy: 0.3792
Epoch 4/5
1563/1563 [==============================] - 94s 60ms/step - loss: 1.9896 - accuracy: 0.4817 - val_loss: 2.4736 - val_accuracy: 0.3855
Epoch 5/5
1563/1563 [==============================] - 86s 55ms/step - loss: 1.9037 - accuracy: 0.5016 - val_loss: 2.4870 - val_accuracy: 0.3946
CPU times: user 5min 37s, sys: 14.5 s, total: 5min 52s
Wall time: 8min 33s
313/313 - 13s - loss: 2.4870 - accuracy: 0.3946


[2.4870386123657227, 0.3946000039577484]